In [47]:
import pandas as pd

## Next Cell is the cell that we need to change for changing data generation

In [48]:
datapath = "../Data/groups/"
#For the next 2 df change the group number to load different data (1 to 6)
group = 6

# Load is the dataframe that represents the load of each building for each scenario and hours of the day
load_df = pd.read_csv(datapath+"group_"+str(group)+"_df_Buildings_t.csv",delimiter=",")
# SolarGain is the dataframe that represents the load of each building for each scenario and hours of the day
solarGain_df = pd.read_csv(datapath+"/group_"+str(group)+"_df_Unit_t.csv")

In [49]:
load_df

,Hub,Period,Time,Domestic_electricity,House_Q_DHW,T_in,House_Q_heating,House_Q_cooling,Th_supply,Th_return,HeatGains,SolarGains
0,Building1,1,1,0.536135,0.000000,20.000000,19.533288,0,45.122733,36.748489,0.857816,0.000000
1,Building1,1,2,1.072270,0.000000,20.000000,20.403640,0,46.242136,37.494757,0.857816,0.000000
2,Building1,1,3,1.072270,0.000000,20.000000,21.398328,0,47.521454,38.347636,0.857816,0.000000
3,Building1,1,4,1.072270,0.000000,20.000000,21.771336,0,48.001198,38.667465,0.857816,0.000000
4,Building1,1,5,1.072270,0.000000,20.000000,22.268680,0,48.640857,39.093905,0.857816,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
72595,Building99,10,22,0.083277,0.000000,28.847874,0.000000,0,20.000000,20.000000,0.047190,0.000000
72596,Building99,10,23,0.083277,0.000000,28.717303,0.000000,0,20.904264,20.602843,0.047190,0.000000
72597,Building99,10,24,0.041639,0.000000,28.576584,0.000000,0,22.023667,21.349111,0.047190,0.000000
72598,Building99,11,1,0.436553,0.402892,0.000000,0.836814,0,64.305117,49.536744,0.047190,0.000000


Then just basic run of the cell to create the data for later processing

In [50]:
datapath = "../Data/"
#Import previous data
scenario_df = pd.read_csv(datapath+"Chaudron_scenarios.csv",delimiter=";")
build_df = pd.read_csv(datapath+"building_data_KNN.csv")

# Add the grid node to which each building is connected:
load_df = load_df.merge(build_df[["Hub","grid_node"]],on="Hub",how="right")


# SolarGain wrangling
# Only select lines that are describing PV and drop useless columns
solarGain_df = solarGain_df[solarGain_df.Unit.str.contains("PV")].drop(columns=["Layer","Units_demand","BAT_E_stored"])
# Extracting the name of the building from Unit:
solarGain_df['Hub'] = solarGain_df['Unit'].str.replace('PV_', '', regex=False)
solarGain_df = solarGain_df.rename(columns={"Units_supply":"PV_production","Units_curtailment":"PV_curtailment"}).drop(columns="Unit").reset_index(drop=True)

# Add the PV production to load_data
load_df = load_df.merge(solarGain_df, on=["Hub","Period","Time"],how="right")

In [51]:
# Get the number of scenarios to iterate
scenarios = scenario_df.Period.to_list()

In [52]:
# create the final dataframe with the active power demand for each bus for each time step for each scenario
final_df = pd.DataFrame()
for sc in scenarios:
    intermediate_df = load_df[load_df.Period == sc].drop(columns=["Hub","Period"]).groupby(["Time","grid_node"]).sum().reset_index()
    intermediate_df["Period"] = sc
    cols = [intermediate_df.columns[-1]] + list(intermediate_df.columns[:-1])
    intermediate_df = intermediate_df[cols]
    final_df = pd.concat([final_df, intermediate_df], axis=0, ignore_index=True)

In [53]:
#Save to csv the final data that will later be used into the allocation
datapath = "../Data/groups/"
final_df.to_csv(datapath+"Clean_demand_632_group"+str(group)+".csv",index=False)

In [54]:
# Vizualisation of the data
final_df.head(60)

,Period,Time,grid_node,Domestic_electricity,House_Q_DHW,T_in,House_Q_heating,House_Q_cooling,Th_supply,Th_return,HeatGains,SolarGains,PV_production,PV_curtailment
0,1,1,1.0,1.281311,0.0,18.000000,11.610407,0.0,42.677792,34.801901,0.570957,0.0,0.0,0.0
1,1,1,2.0,0.755533,0.0,61.469473,28.106231,0.0,137.056644,111.371096,0.515948,0.0,0.0,0.0
2,1,1,5.0,2.366728,0.0,80.420243,103.402160,0.0,181.198007,147.465338,3.786764,0.0,0.0,0.0
3,1,1,6.0,1.723759,0.0,20.000000,62.802626,0.0,45.122733,36.748489,2.758014,0.0,0.0,0.0
4,1,1,10.0,0.022052,0.0,21.105153,0.000000,0.0,46.226013,37.484009,0.000000,0.0,0.0,0.0
5,1,1,11.0,3.460935,0.0,97.765314,145.924825,0.0,168.486210,138.757716,5.375507,0.0,0.0,0.0
6,1,1,13.0,0.643624,0.0,20.040243,28.119889,0.0,45.299502,36.866334,1.029799,0.0,0.0,0.0
7,1,1,16.0,17.196632,0.0,300.458548,742.802780,0.0,680.672948,553.768495,22.525347,0.0,0.0,0.0
8,1,1,17.0,0.284916,0.0,40.000000,12.334163,0.0,90.830631,73.887087,0.429957,0.0,0.0,0.0
9,1,1,19.0,4.502357,0.0,19.352499,94.857005,0.0,45.250289,36.740144,0.941551,0.0,0.0,0.0
